In [34]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")

In [6]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [7]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [8]:
books["tagged_description"].to_csv("tagged_descriptions.txt",
                                   sep="\n",
                                   index=False,
                                   header=False)

In [25]:
raw_documents = TextLoader("tagged_descriptions.txt").load()
# text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
# documents = text_splitter.split_documents(raw_documents)

# Create a document per line
documents = []
for doc in raw_documents:
    lines = doc.page_content.split("\n")
    for line in lines:
        line = line.strip()
        if line:
            documents.append(Document(page_content=line))

In [26]:
documents[0]

Document(metadata={}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and 

In [38]:
huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(
    documents,
    embedding=huggingface_embeddings
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='a8a9d8af-48d1-4bf6-80dd-4d12fc7a9b83', metadata={}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='23ebba3e-3f8b-47eb-b841-99fb4d08d9a7', metadata={}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='cd944109-f7f1-4019-91d1-09ed17092a05', metadata={}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and ba

In [41]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [54]:
# def retrieve_semantic_recommendations(
#         query: str,
#         top_k: int = 10,
# ) -> pd.DataFrame:
#     recs = db_books.similarity_search_with_score(query, k = 50)
#
#     books_list = []
#     for i in range(0, len(recs)):
#         books_list += [int(recs[i].page_content.strip('"').split()[0])]
#
#     return books[books["isbn13"].isin(books_list)].head(top_k)


In [73]:
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    # Step 1: Run semantic similarity search (with score)
    results = db_books.similarity_search_with_score(query, k=50)

    # Step 2: Parse results and extract ISBN + score
    recommendations = []
    seen_isbns = set()

    for doc, score in results:
        # Assume the ISBN is the first token (or improve based on your format)
        isbn_str = doc.page_content.strip('"').split()[0]
        try:
            isbn = int(isbn_str)
        except ValueError:
            continue  # skip bad lines

        if isbn not in seen_isbns:
            recommendations.append((isbn, score))
            seen_isbns.add(isbn)

    # Step 3: Build DataFrame from matching ISBNs and scores
    rec_df = pd.DataFrame(recommendations, columns=["isbn13", "similarity_score"])

    # Step 4: Merge with book metadata
    final_df = rec_df.merge(books, on="isbn13", how="inner")

    # Step 5: Sort by similarity score
    return final_df.sort_values(by="similarity_score", ascending=True).head(top_k)


In [79]:
retrieve_semantic_recommendations("a book about trees")

,isbn13,similarity_score,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780789458209,0.891580,0789458209,Tree,David Burnie,Juvenile Nonfiction,http://books.google.com/books/content?id=Qwsqj...,Photographs and text explore the anatomy and l...,2000.0,4.07,64.0,5.0,Tree,9780789458209 Photographs and text explore the...
1,9780007105045,0.917141,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories..."
2,9780446518628,0.983174,044651862X,The Celestine Prophecy,James Redfield,Fiction,http://books.google.com/books/content?id=UXolx...,You have never read a book like this before --...,1994.0,3.63,247.0,1280.0,The Celestine Prophecy: An Adventure,9780446518628 You have never read a book like ...
3,9780312273224,1.030544,0312273223,Absent in the Spring and Other Novels,Mary Westmacott;Agatha Christie,Fiction,http://books.google.com/books/content?id=rbkRi...,Originally written by Agatha Christie under th...,2001.0,4.19,656.0,88.0,Absent in the Spring and Other Novels: Absent ...,9780312273224 Originally written by Agatha Chr...
4,9781400064892,1.039204,1400064899,The Wild Trees,Richard Preston,Nature,http://books.google.com/books/content?id=IWlU7...,The best-selling author of The Hot Zone takes ...,2007.0,4.08,294.0,5255.0,The Wild Trees: A Story of Passion and Daring,9781400064892 The best-selling author of The H...
5,9780061120077,1.040077,0061120073,A Tree Grows in Brooklyn,Betty Smith,Fiction,http://books.google.com/books/content?id=Y-FZ9...,The beloved American classic about a young gir...,2006.0,4.26,496.0,326733.0,A Tree Grows in Brooklyn,9780061120077 The beloved American classic abo...
6,9780007149124,1.137904,0007149123,Tales from the Perilous Realm,John Ronald Reuel Tolkien,"Fairy tales, English",http://books.google.com/books/content?id=Wla7N...,"Never before published in a single volume, Tol...",2002.0,4.08,178.0,2964.0,Tales from the Perilous Realm,9780007149124 Never before published in a sing...
7,9781590170311,1.149121,1590170318,Witch Grass,Raymond Queneau;Barbara Wright,Fiction,http://books.google.com/books/content?id=sz6A0...,"Seated in a Paris café, a man glimpses another...",2003.0,3.94,328.0,320.0,Witch Grass,"9781590170311 Seated in a Paris café, a man gl..."
8,9780930588793,1.151724,0930588797,Life Amongst the Modocs,Joaquin Miller,Biography & Autobiography,http://books.google.com/books/content?id=6_JEy...,"Miller's masterful book, based on his years du...",1996.0,4.32,456.0,19.0,Life Amongst the Modocs: Unwritten History,"9780930588793 Miller's masterful book, based o..."
9,9780060959036,1.168483,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
